In [0]:
import pandas as pd   
import re
import numpy as np
import matplotlib.pyplot as plt
import os   

In [2]:
from google.colab import drive
#drive.flush_and_unmount()
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!wget https://sota-ydata.s3.amazonaws.com/Paraphrase.csv

--2020-06-14 17:07:20--  https://sota-ydata.s3.amazonaws.com/Paraphrase.csv
Resolving sota-ydata.s3.amazonaws.com (sota-ydata.s3.amazonaws.com)... 52.217.18.116
Connecting to sota-ydata.s3.amazonaws.com (sota-ydata.s3.amazonaws.com)|52.217.18.116|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 44220 (43K) [text/csv]
Saving to: ‘Paraphrase.csv’

Paraphrase.csv      100%[===================>]  43.18K   235KB/s    in 0.2s    

2020-06-14 17:07:21 (235 KB/s) - ‘Paraphrase.csv’ saved [44220/44220]



In [0]:
data = pd.read_csv("Paraphrase.csv")

In [5]:
data

,text_1,text_2,label
0,take measures in order to,take additional measures to,4.000
1,was seeking to,will strive to,3.000
2,canada aims to,canada works to,2.333
3,refugees and,refugee population,2.667
4,measures and,"instruments , and",3.000
...,...,...,...
993,unless specified otherwise,unless specifically stated otherwise,4.667
994,its efforts to combat,the area of combating,3.667
995,considered by the,give consideration to the,4.333
996,an enlarged security,an expanded security,4.000


#ROUGE metric

https://rxnlp.com/how-rouge-works-for-evaluation-of-summarization-tasks/#.XuZUKvkzZaQ

https://pypi.org/project/easy-rouge/

https://pypi.org/project/py-rouge/

In [6]:
!pip install easy-rouge

In [0]:
from rouge.rouge import rouge_n_sentence_level
from rouge.rouge import rouge_l_sentence_level

In [19]:
summary_sentence = 'the capital of China is Beijing'.split()
reference_sentence = 'Beijing is the capital of China'.split()

# Calculate ROUGE-2.
recall, precision, rouge = rouge_n_sentence_level(summary_sentence, reference_sentence, 2)
print('ROUGE-2-R', recall)
print('ROUGE-2-P', precision)
print('ROUGE-2-F', rouge)

recall, precision, rouge = rouge_l_sentence_level(summary_sentence, reference_sentence)
print('ROUGE-l-R', recall)
print('ROUGE-l-P', precision)
print('ROUGE-l-F', rouge)



ROUGE-2-R 0.6
ROUGE-2-P 0.6
ROUGE-2-F 0.6
ROUGE-l-R 0.6666666666666666
ROUGE-l-P 0.6666666666666666
ROUGE-l-F 0.6666666666666666


In [0]:
recall_1_list = []
recall_2_list = []
precision_1_list = []
precision_2_list = []
F_1_list = []
F_2_list = []

recall_l_list = []
precision_l_list = []
F_l_list = []

i = 0
while i < len(data):
    sentence_ori = data.iloc[i]['text_1']
    sentence_gen = data.iloc[i]['text_2']

    sentence_ori = sentence_ori.split()
    sentence_gen = sentence_gen.split()

    # Calculate ROUGE-1
    recall_1, precision_1, rouge_1 = rouge_n_sentence_level(sentence_gen, sentence_ori, 1)
    # Calculate ROUGE-2
    recall_2, precision_2, rouge_2 = rouge_n_sentence_level(sentence_gen, sentence_ori, 2)
    # Calculate ROUGE-l
    recall_l, precision_l, rouge_l = rouge_l_sentence_level(sentence_gen, sentence_ori)

    recall_1_list.append(recall_1)
    recall_2_list.append(recall_2)
    recall_l_list.append(recall_l)
    precision_1_list.append(precision_1)
    precision_2_list.append(precision_2)
    precision_l_list.append(precision_l)
    F_1_list.append(rouge_1)
    F_2_list.append(rouge_2)
    F_l_list.append(rouge_l)
    i+=1

In [0]:
new_data = data.copy() 
new_data['ROUGE-1 recall'] = recall_1_list
new_data['ROUGE-1 precision'] = precision_1_list
new_data['ROUGE-1 F'] = F_1_list
new_data['ROUGE-2 recall'] = recall_2_list
new_data['ROUGE-2 precision'] = precision_2_list
new_data['ROUGE-2 F'] = F_2_list
new_data['ROUGE-L recall'] = recall_l_list
new_data['ROUGE-L precision'] = precision_l_list
new_data['ROUGE-L F'] = F_l_list

In [22]:
new_data

,text_1,text_2,label,ROUGE-1 recall,ROUGE-1 precision,ROUGE-1 F,ROUGE-2 recall,ROUGE-2 precision,ROUGE-2 F,ROUGE-L recall,ROUGE-L precision,ROUGE-L F
0,take measures in order to,take additional measures to,4.000,0.600000,0.750000,0.666667,0.0,0.0,0.0,0.600000,0.750000,0.666667
1,was seeking to,will strive to,3.000,0.333333,0.333333,0.333333,0.0,0.0,0.0,0.333333,0.333333,0.333333
2,canada aims to,canada works to,2.333,0.666667,0.666667,0.666667,0.0,0.0,0.0,0.666667,0.666667,0.666667
3,refugees and,refugee population,2.667,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
4,measures and,"instruments , and",3.000,0.500000,0.333333,0.400000,0.0,0.0,0.0,0.500000,0.333333,0.400000
...,...,...,...,...,...,...,...,...,...,...,...,...
993,unless specified otherwise,unless specifically stated otherwise,4.667,0.666667,0.500000,0.571429,0.0,0.0,0.0,0.666667,0.500000,0.571429
994,its efforts to combat,the area of combating,3.667,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
995,considered by the,give consideration to the,4.333,0.333333,0.250000,0.285714,0.0,0.0,0.0,0.333333,0.250000,0.285714
996,an enlarged security,an expanded security,4.000,0.666667,0.666667,0.666667,0.0,0.0,0.0,0.666667,0.666667,0.666667


In [0]:
new_data.to_csv('Paraphrase_with_ROUGE.csv', index=False)

In [0]:
!cp Paraphrase_with_ROUGE.csv "/content/drive/My Drive/Data Science Course/semester 2/NLP/homework/HW3 - semantic similarity"